In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

import pandas as pd 

# importing functions
import sys
sys.path.append('/Users/audreyburggraf/Desktop/THESIS/Functions')

from functions import *
from unit_conversion_functions import *
from fitting_functions import *
from printing_functions import *

In [2]:
# names 
# inj_params_0P: array holding the injected parameters for the 0 planet fit 
# inj_params_1P: array holding the injected parameters for the 1 planet fit 

In [3]:
df = pd.read_csv('/Users/audreyburggraf/Desktop/THESIS/Gaia Data Files/gaia_data.csv')

#  add the absolute gaia K band magnitude and stellar mass
df["M_ks"] = df.ks_m + 5 + 5*np.log10(df.parallax/1000)

# cut off certain range 
df = df[(4 < df.M_ks) & (df.M_ks < 11)].reset_index(drop=True)

# set the stellar mass 
df["stellar_mass"] =  find_star_mass(df.M_ks)

In [4]:
N1 = 70
N2 = 70000

seed = 8

# call function
parameter_result, synthetic_result, model_result, error_result = find_signal_components(df, N1, N2, print_params='both')

# break down the result statement 

# parameter_result includes the no planet and 1 planet parameters in their proper units 
(inj_params_0P, inj_params_1P) = parameter_result

# model_results includes the signal components and times for the synthetic in [uas] and [years]
(prop_ra_synthetic, prop_dec_synthetic, 
 parallax_ra_synthetic, parallax_dec_synthetic, 
 planetary_ra_synthetic, planetary_dec_synthetic, 
 times_synthetic) = synthetic_result

# model_results includes the signal components and times for the model in [uas] and [years]
(prop_ra_model, prop_dec_model, 
 parallax_ra_model, parallax_dec_model, 
 planetary_ra_model, planetary_dec_model, 
 times_model) = model_result

# error_result includes noise in ra and dec direction plus error (same for ra and dec) in [uas]
(noise_ra, noise_dec, errors) = error_result

N for synthetic data:  70
N for model data    :  70000
 
Planetary parameters:
e           : 0.302912000589295  [unitless]
omega       : 4.313677832870261  [radians]
Omega       : 2.6435535652811666  [radians]
cos_i       : 0.12912894817108467  [unitless]
m_planet    : 0.9552856398248216  [Jupiter masses]
log10(P_orb): 0.0405557069420337  [log10(years)]
t_peri      : 0.3334655018688819  [years]
 
Gaia parameters:
alpha0      : 1  [degrees]
delta0      : 2  [degrees]
mu_alpha    : 63.20071839452851  [mas/year]
mu_delta    : -81.25858742746935  [mas/year]
parallax    : 50  [mas]
m_star      : 0.1533814175442969  [M_sun]
x =  202554
 
astrometric_signature( [0.95528564] [0.15338142] [0.04055571] [20.] ) =  [169.34622012] [uas]


In [5]:
signal_ra_obs  = prop_ra_synthetic + parallax_ra_synthetic + planetary_ra_synthetic + noise_ra
signal_dec_obs = prop_dec_synthetic + parallax_dec_synthetic + planetary_dec_synthetic + noise_dec

In [6]:
fitted_params_0P, np_chi_sq = no_planet_fit(inj_params_0P, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_0P, fitted_params_0P, return_type='np')

alpha0  : real=1.00000, fitted=1.00599, difference=0.00599
delta0  : real=2.00000, fitted=1.99661, difference=0.00339
mu_alpha: real=63.20072, fitted=63.19964, difference=0.00108
mu_delta: real=-81.25859, fitted=-81.25577, difference=0.00282
parallax: real=50.00000, fitted=49.99719, difference=0.00281


In [7]:
fitted_params_1P, wp_chi_sq = one_planet_fit(inj_params_1P, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_1P, fitted_params_1P, return_type='wp')

alpha0      : real=1.00000, fitted=1.00173, difference=0.00173
delta0      : real=2.00000, fitted=2.00170, difference=0.00170
mu_alpha    : real=63.20072, fitted=63.20094, difference=0.00022
mu_delta    : real=-81.25859, fitted=-81.25639, difference=0.00220
parallax    : real=50.00000, fitted=49.97702, difference=0.02298
e           : real=0.30291, fitted=0.81723, difference=0.51432
omega       : real=4.31368, fitted=7.70971, difference=3.39604
Omega       : real=2.64355, fitted=1.15582, difference=1.48773
cos_i       : real=0.12913, fitted=0.00356, difference=0.12557
m_planet    : real=0.95529, fitted=0.02845, difference=0.92684
log10(P_orb): real=0.04056, fitted=0.01447, difference=0.02609
t_peri      : real=0.33347, fitted=0.31624, difference=0.01723


/Users/audreyburggraf/Desktop/THESIS/Functions/functions.py:472: RuntimeWarning: invalid value encountered in double_scalars
  a = (P_orb**2 * m_star)**(1/3)


In [8]:
detection_result = detection_function(np_chi_sq, wp_chi_sq, inj_params_1P, fitted_params_1P, N1)
detection_result

(155.00892437562246, 177.75830198867925, 22.749377613056794, array([0]))